In [ ]:
#root分割adaptive
#全ての実験の結果を入れるリスト
clt_opt_results_e_s=[]
#平均二乗誤差を入れるリスト
ave_sq_error_e_s=[]
#実験回数の設定
ex_it=100
#実験イテレーション
for r in range(ex_it):
    #振幅
    a=0.2
    #depolarization確率
    p=0.9
    pi=np.pi
    theta=np.arcsin(np.sqrt(a))
    #1回あたりの実験結果を入れるリスト
    opt_results_e_s=[]
    #測定結果を入れるリスト
    n0=[]
    n1=[]
    n2=[]
    #初期値
    theta_0=[pi/4]
    #増幅した位相の係数
    Ng=[]
    #増幅回数
    m=[]
    #回路内で作用する量子ビットリスト
    qlist=[]
    #増幅あたりの分割数
    it=2
    #shot数
    N=40000
    #qbit数
    n=4
    #最大増幅数
    amf=50
    #作用する量子ビット
    for q in range(n):
        qlist.append(q)
    #量子系次元
    d=2**n
    #測定する量子ビット
    N0='0'*n
    N1='1'*n
    #増幅測定イテレーション
    for k in range(amf):
        #増幅した係数
        Ng.append(2*k+1)
        #増幅数
        m.append(k+1)
        n0.append([])
        n1.append([])
        n2.append([])
        #回路内行列の構成
        R_1=[np.cos(Ng[k]*theta),-np.sin(Ng[k]*theta)]
        R_2=[np.sin(Ng[k]*theta),np.cos(Ng[k]*theta)]
        R_1[1:1]=[0]*(d-2)
        R_2[1:1]=[0]*(d-2)
        R_n=[R_1,R_2]
        for e in range(d-2):
            Z=[0]*(d-1)
            Z.insert(e+1,1)
            R_n.insert(-1,Z)
        R_en_n=Operator(R_n)
        for i in range(it):
            #回路内測定用行列構成
            theta_e=Ng[k]*theta_0[i+k*it]+pi/4
            R_1e=[np.cos(-theta_e),-np.sin(-theta_e)]
            R_2e=[np.sin(-theta_e),np.cos(-theta_e)]
            R_1e[1:1]=[0]*(d-2)
            R_2e[1:1]=[0]*(d-2)
            R_ne=[R_1e,R_2e]
            for f in range(d-2):
                Ze=[0]*(d-1)
                Ze.insert(f+1,1)
                R_ne.insert(-1,Ze)
            R_en_ne=Operator(R_ne)
            circ_e_n=QuantumCircuit(n)
            circ_e_n.unitary(R_en_n,qlist,label='R_4')
            circ_e_n.unitary(R_en_ne,qlist,label='R_4_e')
            circ_e_n.measure_all()
            # Create an empty noise model
            noise_depolarizing = NoiseModel()
            noise_depolarizing.add_basis_gates(['unitary'])
            # Add depolarizing error to all single qubit u1, u2, u3 gates
            error = depolarizing_error(1-p**m[k], n)
            noise_depolarizing.add_all_qubit_quantum_error(error, 'R_4')
            # Create noisy simulator backend
            sim_noise = AerSimulator(noise_model=noise_depolarizing)
            # Transpile circuit for noisy basis gates
            circ_e_n_tnoise = transpile(circ_e_n, sim_noise)
            #測定の切り替え
            if i==0:
                s=np.sqrt(N)
                result_noise = sim_noise.run(circ_e_n_tnoise,shots=s).result()
            else:
                s=N-np.sqrt(N)
                result_noise = sim_noise.run(circ_e_n_tnoise,shots=s).result()
            counts_noise = result_noise.get_counts(0)
            #測定結果の記録
            n0[k].append(counts_noise[N0])
            n1[k].append(counts_noise[N1])
            n2[k].append(s-n0[k][i]-n1[k][i])
            #パラーメータの推定
            def func(x):
                ans=0
                for l in range(k+1):
                    for j in range(i+1):
                        angle=x[0]-theta_0[j+l*it]
                        ans+=np.log((p**m[l])*(1+np.sin(2*Ng[l]*angle))/2+(1-p**m[l])/d)*n0[l][j]
                        ans+=np.log((p**m[l])*(1-np.sin(2*Ng[l]*angle))/2+(1-p**m[l])/d)*n1[l][j]
                        ans+=np.log((d-2)*(1-p**m[l])/d)*n2[l][j]
                return -ans
            x=opt.brute(func,[(0,pi/2)])
            #推定結果の記録
            opt_results_e_s.append(np.sin(x[0])**2)
            theta_0.append(x[0])
    #平均二乗誤差の計算記録
    ave_sq_error_e_s.append(sum((opt_results_e_s[2*n+1]-a)**2 for n in range(int(len(opt_results_e_s)/2)))*2/len(opt_results_e_s))
    #実験結果の記録
    clt_opt_results_e_s.append(opt_results_e_s)
print(ave_sq_error_e_s)
circ_e_n.draw('mpl')

In [ ]:
#計算基底測定版
clt_opt_results_c=[]
ave_sq_error_c=[]
ex_it=100
for r in range(ex_it):
    a=0.2
    p=0.9
    pi=np.pi
    theta=np.arcsin(np.sqrt(a))
    opt_results_c=[]
    n0=[]
    n1=[]
    n2=[]
    Ng=[]
    m=[]
    qlist=[]
    it=1
    N=40000
    s=N-np.sqrt(N)
    n=4
    amf=50
    for q in range(n):
        qlist.append(q)
    d=2**n
    N0='0'*n
    for k in range(amf):
        Ng.append(2*k+1)
        m.append(k+1)
        n0.append([])
        n1.append([])
        n2.append([])
        R_1=[np.cos(Ng[k]*theta),-np.sin(Ng[k]*theta)]
        R_2=[np.sin(Ng[k]*theta),np.cos(Ng[k]*theta)]
        R_1[1:1]=[0]*(d-2)
        R_2[1:1]=[0]*(d-2)
        R_n=[R_1,R_2]
        for e in range(d-2):
            Z=[0]*(d-1)
            Z.insert(e+1,1)
            R_n.insert(-1,Z)
        R_en_n=Operator(R_n)
        for i in range(it):
            circ_c_n=QuantumCircuit(n)
            circ_c_n.unitary(R_en_n,qlist,label='R_4')
            circ_c_n.measure_all()
            # Create an empty noise model
            noise_depolarizing = NoiseModel()
            noise_depolarizing.add_basis_gates(['unitary'])
            # Add depolarizing error to all single qubit u1, u2, u3 gates
            error = depolarizing_error(1-p**m[k], n)
            noise_depolarizing.add_all_qubit_quantum_error(error, 'R_4')
            # Create noisy simulator backend
            sim_noise = AerSimulator(noise_model=noise_depolarizing)
            # Transpile circuit for noisy basis gates
            circ_c_n_tnoise = transpile(circ_c_n, sim_noise)
            # Run and get counts
            result_noise = sim_noise.run(circ_c_n_tnoise,shots=s).result()
            counts_noise = result_noise.get_counts(0)
            n0[k].append(counts_noise[N0])
            n1[k].append(N-n0[k][i])
            def func(x):
                ans=0
                for l in range(k+1):
                    for j in range(i+1):
                        angle=x[0]
                        ans+=np.log((p**m[l])*(np.cos(Ng[l]*angle)**2)+(1-p**m[l])/d)*n0[l][j]
                        ans+=np.log((p**m[l])*(np.sin(Ng[l]*angle)**2)+((d-1)*(1-p**m[l]))/d)*n1[l][j]
                return -ans
            x=opt.brute(func,[(0,pi/2)])
            opt_results_c.append(np.sin(x[0])**2)
    ave_sq_error_c.append(sum((n-a)**2 for n in opt_results_c)/len(opt_results_c))
    clt_opt_results_c.append(opt_results_c)
print(ave_sq_error_c)
circ_c_n.draw('mpl')

In [ ]:
#1:1分割版adaptive
clt_opt_results_e=[]
ave_sq_error_e=[]
ex_it=100
for r in range(ex_it):
    a=0.2
    p=0.9
    pi=np.pi
    theta=np.arcsin(np.sqrt(a))
    opt_results_e=[]
    n0=[]
    n1=[]
    n2=[]
    theta_0=[pi/4]
    Ng=[]
    m=[]
    qlist=[]
    it=2
    N=40000
    s=int((N-np.sqrt(N))/2)
    n=4
    amf=50
    for q in range(n):
        qlist.append(q)
    d=2**n
    N0='0'*n
    N1='1'*n
    for k in range(amf):
        Ng.append(2*k+1)
        m.append(k+1)
        n0.append([])
        n1.append([])
        n2.append([])
        R_1=[np.cos(Ng[k]*theta),-np.sin(Ng[k]*theta)]
        R_2=[np.sin(Ng[k]*theta),np.cos(Ng[k]*theta)]
        R_1[1:1]=[0]*(d-2)
        R_2[1:1]=[0]*(d-2)
        R_n=[R_1,R_2]
        for e in range(d-2):
            Z=[0]*(d-1)
            Z.insert(e+1,1)
            R_n.insert(-1,Z)
        R_en_n=Operator(R_n)
        for i in range(it):
            theta_e=Ng[k]*theta_0[i+k*it]+pi/4
            R_1e=[np.cos(-theta_e),-np.sin(-theta_e)]
            R_2e=[np.sin(-theta_e),np.cos(-theta_e)]
            R_1e[1:1]=[0]*(d-2)
            R_2e[1:1]=[0]*(d-2)
            R_ne=[R_1e,R_2e]
            for f in range(d-2):
                Ze=[0]*(d-1)
                Ze.insert(f+1,1)
                R_ne.insert(-1,Ze)
            R_en_ne=Operator(R_ne)
            circ_e_n=QuantumCircuit(n)
            circ_e_n.unitary(R_en_n,qlist,label='R_4')
            circ_e_n.unitary(R_en_ne,qlist,label='R_4_e')
            circ_e_n.measure_all()
            # Create an empty noise model
            noise_depolarizing = NoiseModel()
            noise_depolarizing.add_basis_gates(['unitary'])
            # Add depolarizing error to all single qubit u1, u2, u3 gates
            error = depolarizing_error(1-p**m[k], n)
            noise_depolarizing.add_all_qubit_quantum_error(error, 'R_4')
            # Create noisy simulator backend
            sim_noise = AerSimulator(noise_model=noise_depolarizing)
            # Transpile circuit for noisy basis gates
            circ_e_n_tnoise = transpile(circ_e_n, sim_noise)
            # Run and get counts
            result_noise = sim_noise.run(circ_e_n_tnoise,shots=s).result()
            counts_noise = result_noise.get_counts(0)
            n0[k].append(counts_noise[N0])
            n1[k].append(counts_noise[N1])
            n2[k].append(N-n0[k][i]-n1[k][i])
            def func(x):
                ans=0
                for l in range(k+1):
                    for j in range(i+1):
                        angle=x[0]-theta_0[j+l*it]
                        ans+=np.log((p**m[l])*(1+np.sin(2*Ng[l]*angle))/2+(1-p**m[l])/d)*n0[l][j]
                        ans+=np.log((p**m[l])*(1-np.sin(2*Ng[l]*angle))/2+(1-p**m[l])/d)*n1[l][j]
                        ans+=np.log((d-2)*(1-p**m[l])/d)*n2[l][j]
                return -ans
            x=opt.brute(func,[(0,pi/2)])
            opt_results_e.append(np.sin(x[0])**2)
            theta_0.append(x[0])
    ave_sq_error_e.append(sum((n-a)**2 for n in opt_results_e)/len(opt_results_e))
    clt_opt_results_e.append(opt_results_e)
print(ave_sq_error_e)
circ_e_n.draw('mpl')